# Some preliminary checks

In [1]:
import torch
import os

#os.environ["RAY_DEDUP_LOGS"] = "0"

import ray 

print("Ray version :", ray.__version__)

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())

print("MPS Available:", torch.backends.mps.is_available())

torch._dynamo.list_backends()

Ray version : 2.39.0
PyTorch Version: 2.5.1
CUDA Available: False
MPS Available: True


['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'tvm']

### Important : Number of CPUs and GPUs available

In [2]:
import psutil

# print number of gpus / CPUs
print("Number of CPUs: ", psutil.cpu_count())

num_cpus = 12
num_gpus = 0
num_learner = 1

assert num_cpus <= psutil.cpu_count()

Number of CPUs:  12


# Environement and algorithm configuration

Some of the commented lines are preparation work to use a futur feature of RLLib

Note: In multi-agent environments, `rollout_fragment_lenght` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.


In [3]:
from ray.rllib.policy.policy import PolicySpec
#from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec
#from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec

from ray.tune.registry import get_trainable_cls

from particle_2d_env import Particle2dEnvironment
from particle_2d_env import MetricsCallbacks, RenderingCallbacks
from config import run_config

ALGO = "PPO"        
FRAMEWORK= "torch" # always "torch"
env = Particle2dEnvironment(run_config["env"])

config = (
    get_trainable_cls(ALGO).get_default_config()
    .environment(Particle2dEnvironment, env_config=run_config["env"])
    .framework(FRAMEWORK,)
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    #.callbacks(MetricsCallbacks)
    #.callbacks(RenderingCallbacks)
    # Specify the learner's hyperparameters.
    .training(
        num_sgd_iter=5,          
        num_epochs=10,
        train_batch_size_per_learner=512,             # the number of step collected
        model={
            "fcnet_hiddens": [128, 128, 128], 
            "use_attention": True,
            #"use_lstm": False,
            #"max_seq_len": 5,
            #"lstm_cell_size": 16,
        },
        #lr=tune.grid_search([0.01, 0.001, 0.0001])
    )
    .multi_agent(
        policies= {
            "prey": PolicySpec(
                policy_class=None,  # infer automatically from Algorithm
                observation_space=env.observation_space[0],  # if None infer automatically from env
                action_space=env.action_space[0],  # if None infer automatically from env
                config={"gamma": 0.85},  # use main config plus <- this override here
            ),
            "predator": PolicySpec(
                policy_class=None,
                observation_space=env.observation_space[0],
                action_space=env.action_space[0],
                config={"gamma": 0.85},
            ),
        },
        policy_mapping_fn = lambda id, *arg, **karg: "prey" if env.particule_agents[id].agent_type == 0 else "predator",
        policies_to_train=["prey", "predator"],
        count_steps_by="agent_steps",
    )
    .learners(
        num_learners=4,  # or >2
        num_cpus_per_learner=1,  # <- default
        num_gpus_per_learner=0,  # <- default
    )
    .resources(num_cpus_for_main_process=1)  # default is 1
    .env_runners(
        rollout_fragment_length="auto", #"auto" for PPO explained here : https://docs.ray.io/en/latest/rllib/rllib-sample-collection.html
        batch_mode= 'truncate_episodes',
        num_env_runners=1, # need 2 for IMPALA, 1 for PPO
        num_envs_per_env_runner=1,
    )
    .checkpointing(export_native_model_files=True)
)


/opt/homebrew/Caskroom/miniforge/base/envs/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/collective_env/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/collective_env/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not withi

# Training

In [4]:
from ray.rllib.policy.policy import Policy
from ray.rllib.algorithms.callbacks import DefaultCallbacks

path_to_checkpoint = None #os.getcwd() + "/ray_results" + "PPO_2024-05-18_00-08-19/PPO_Particle2dEnvironment_bb60c_00000_0_2024-05-18_00-08-19/checkpoint_000001"

def restore_weights(path, policy_type):
    checkpoint_path = os.path.join(path, f"policies/{policy_type}")
    restored_policy = Policy.from_checkpoint(checkpoint_path)
    return restored_policy.get_weights()

if path_to_checkpoint is not None: 
    class RestoreWeightsCallback(DefaultCallbacks):
        def on_algorithm_init(self, *, algorithm: "Algorithm", **kwargs) -> None:
            algorithm.set_weights({"predator": restore_weights(path_to_checkpoint, "predator")})
            algorithm.set_weights({"prey": restore_weights(path_to_checkpoint, "prey")})

    config.callbacks(RestoreWeightsCallback)



## Launch training

In [ ]:
from ray import train, tune
from ray.tune import Tuner
from ray.air.integrations.wandb import WandbLoggerCallback
from ray.rllib.utils.test_utils import check_learning_achieved
import os

ray.init(
    num_cpus=num_cpus, 
    num_gpus=num_gpus
)

# Stop criterium
stop = {  
    "training_iteration": 1500,
    #"timesteps_total": 200000000,
}

# Read the API key from the file to use Wanddb
with open('wandb_api_key.txt', 'r') as file:
    api_key = file.read().strip()
callbacks = [
    WandbLoggerCallback(                   
        project="marl-rllib", 
        group=ALGO,
        api_key=api_key,
        log_config=True,
        upload_checkpoints=True
    ), 
]

# When to save the models 
checkpoint_config = train.CheckpointConfig(         
    checkpoint_at_end=True,
    checkpoint_frequency=10,
)

# Where to save 
# absolute path + ray_results directory
storage_path=os.getcwd() + "/ray_results"

if path_to_checkpoint is None : 
    tuner = tune.Tuner(
        ALGO,                                                 # Defined before
        param_space=config,                                   # Defined before
        run_config=train.RunConfig(    
            storage_path=storage_path,
            stop=stop,
            verbose=3,
            callbacks=callbacks,
            checkpoint_config=checkpoint_config,
        ),
    )
    # Run the experiment 
    results = tuner.fit()

# If we instantiate previously trained neural network
else: 
    callbacks.append(RestoreWeightsCallback)

    results = tune.run(
        ALGO,
        config=config.to_dict(),
        storage_path=storage_path,
        stop=stop,
        verbose=3,
        callbacks=callbacks,
        checkpoint_config=checkpoint_config,
    )


ray.shutdown()


2024-12-01 22:02:25,152	INFO worker.py:1819 -- Started a local Ray instance.
2024-12-01 22:02:25,794	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-12-01 22:02:25,811	WARNING algorithm_config.py:4484 -- You configured a custom `model` config (probably through calling config.training(model=..), whereas your config uses the new API stack! In order to switch off the new API stack, set in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. If you DO want to use the new API stack, configure your model, instead, through: `config.rl_module(model_config={..})`.
2024-12-01 22:02:25,813	WARNING ppo.py:305 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2024-12-01 22:02:25,813	WARNING algorithm_config.py:4484 -- You configured a custom `model` config (probably through calling 

(PPO pid=34265) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/tanguy/Code/Finebouche/collective_behavior/ray_results/PPO_2024-12-01_22-02-25/PPO_Particle2dEnvironment_91321_00000_0_2024-12-01_22-02-25/checkpoint_000000)
(_WandbLoggingActor pid=34296) wandb: Adding directory to artifact (/Users/tanguy/Code/Finebouche/collective_behavior/ray_results/PPO_2024-12-01_22-02-25/PPO_Particle2dEnvironment_91321_00000_0_2024-12-01_22-02-25/checkpoint_000000)... 
(_WandbLoggingActor pid=34296) Done. 0.2s
